In [ ]:
!pip install langchain 
!pip install gpt4all
!pip install PyPDF2

In [ ]:
import PyPDF2
import os

In [ ]:
if "Data_text_files" not in os.listdir():
    os.mkdir('Data_text_files')

In [ ]:
def create_docs(curr_dir = '.'):
    for file in os.listdir(curr_dir):
        if '.pdf' in file:
            pdfile = PyPDF2.PdfReader(open(curr_dir + '/' + file, 'rb'))
            with open('./Data_text_files/' + file.replace('.pdf', '.txt'), 'w', encoding='utf-8') as dest:
                contents = []
                for i in range(len(pdfile.pages)):
                    contents.append(pdfile.pages[i].extract_text())
                dest.write('\n'.join(contents))


In [ ]:
create_docs('./Data/')

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 20,
    length_function = len
)

In [ ]:
texts = text_splitter.create_documents([open('./Data_text_files/' + i, encoding = 'utf-8').read() for i in os.listdir('./Data_text_files/')])

In [ ]:
len(texts)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
store = FAISS.from_documents(texts, embeddings)

In [ ]:
import pickle

In [ ]:
pickle.dump(store, open('./store.pkl', 'wb'))

In [ ]:
#Using LLM from here

In [ ]:
store = pickle.load(open('./store.pkl', 'rb'))

In [ ]:
import gpt4all

In [ ]:
if "ggml-model-gpt4all-falcon-q4_0.bin" in os.listdir():
    llm = gpt4all.GPT4All(model_name='ggml-model-gpt4all-falcon-q4_0', model_path='.')
else:
    llm = gpt4all.GPT4All("ggml-model-gpt4all-falcon-q4_0")

In [ ]:
def generate_qa(query):
    docs = store.similarity_search(query, 2)
    context = ""
    for i in docs:
        context += (i.page_content + '\n\n') 
    return llm.generate(f"Use the context to generate questions for an exam. The questions need to be of descriptive type\n\n {context}")
    